In [1]:
import sys
sys.path.append('..')

In [2]:
from tofina.components.backtest import Backtester
import tofina.components.instrument as instrument
from tofina.extern.arch import forecastDecoratorGARCH
from tofina.extern.yfinance import loadHistoricalStockData
import datetime as dt
import numpy as np
from pathlib import Path
import shutil
import pandas as pd
from copy import deepcopy

SPY = loadHistoricalStockData("SPY")
start_date = dt.datetime(2023, 1, 1)
end_date = dt.datetime(2023, 1, 14)
#end_date = dt.datetime(2023, 1, 4)
split_date = start_date - dt.timedelta(days=1)
timestamps = SPY.index[
    np.logical_and(SPY.index >= start_date, SPY.index < end_date)
]
horizon = 20
simulations = 1000
arch_kwargs = dict(vol="Garch", p=3, o=0, q=3, dist="t", mean="AR", lags=5)
forecaster = forecastDecoratorGARCH(
    SPY, split_date, horizon, simulations, **arch_kwargs
)
backtester = Backtester(timestamps=timestamps)
backtester.stockDataFromDataFrame(SPY, "SPY")
backtester.registerForecaster(forecaster, ["SPY"])
backtester.addDeposit(interestRate=0.2/252)
backtester.addIntstrumentToPortfolio("SPY", "Stock", 
            instrument.NonDerivativePayout
        )
backtester.addIntstrumentToPortfolio(
        "SPY", "Stock Short", instrument.NonDerivativePayoutShort
    )
df_path = "/Users/andriylevitskyy/Desktop/tofina/tests/data/options/SPY.csv"
df = pd.read_csv(df_path)
df["Date"] = df["Date"].apply(lambda x: x.replace(" ", ""))
df["Expiry Date"] = df["Expiry Date"].apply(lambda x: x.replace(" ", ""))
df["Date"] = pd.to_datetime(df["Date"])
df["Expiry Date"] = pd.to_datetime(df["Expiry Date"])
df = df[df["Date"] >= start_date]
df = df[df["Date"] < end_date]
backtester.parseOptionData(df, "SPY", sampleOption=100)

dir_path = Path("./results/SPY_GARCH_Options")
if dir_path.exists():
    shutil.rmtree(dir_path)
dir_path.mkdir(parents=True, exist_ok=False)
backtester.optimizeStrategy(dir_path, RiskAversion=0.9, stop=False, iterations = 5000)
backtesterCopy = deepcopy(backtester)
results = backtester.evaluateStrategy()

[*********************100%%**********************]  1 of 1 completed


Iteration:      5,   Func. Count:     94,   Neg. LLF: 30356.489405585766
Iteration:     10,   Func. Count:    181,   Neg. LLF: 15244.176184581413
Iteration:     15,   Func. Count:    263,   Neg. LLF: 10144.559858087046
Iteration:     20,   Func. Count:    344,   Neg. LLF: 10075.081632452344
Iteration:     25,   Func. Count:    420,   Neg. LLF: 10073.148971941075
Optimization terminated successfully    (Exit mode 0)
            Current function value: 10073.14896934825
            Iterations: 27
            Function evaluations: 449
            Gradient evaluations: 27


/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
100it [00:00, 585.94it/s]
100it [00:00, 603.40it/s]
100it [00:00, 600.73it/s]
100it [00:00, 665.55it/s]
100it [00:00, 615.00it/s]
100it [00:00, 678.99it/s]
100it [00:00, 653.00it/s]
100it [00:00, 725.18it/s]
100it [00:00, 678.62it/s]
100%|██████████| 9/9 [04:23<00:00, 29.23s/it]


In [10]:
for ts in timestamps:
    print(ts, float(results[ts]["real"].sum(axis=1)), float(results[ts]["simulation"].sum(axis=1).mean()))


2023-01-03 00:00:00 0.2042976282874788 0.06989371854983861
2023-01-04 00:00:00 0.17764539462041948 0.0477392247346816
2023-01-05 00:00:00 0.332041732191257 0.048190974722344945
2023-01-06 00:00:00 0.003929465524986742 0.05190422761790645
2023-01-09 00:00:00 0.11333887329420433 0.0575807841789145
2023-01-10 00:00:00 0.3092132546569198 0.10078679208924377
2023-01-11 00:00:00 0.08576951952041141 0.019732227616336976
2023-01-12 00:00:00 0.23068633138379885 0.15248244000343453
2023-01-13 00:00:00 0.3275985539109829 0.11955563115045659


In [9]:
timestamps =  list(backtester.pointInTimePortfolio.keys())
testPortfolio = backtester.pointInTimePortfolio[timestamps[0]]

In [19]:
weights = backtester.pointInTimePortfolio[timestamps[0]].strategy.normalizedWeights
instruments = backtester.pointInTimePortfolio[timestamps[0]].strategy.instruments.keys()
prices = []
for i in instruments:
    prices.append(backtester.pointInTimePortfolio[timestamps[0]].strategy.instruments[i].price)
weightsDict = {}
for w,i,p in zip(weights,instruments, prices):
    weightsDict[i] = {"weight":float(w), "price":float(p)}
pd.DataFrame(weightsDict).T.sort_values(by="weight", ascending=False)

,weight,price
Deposit_GIC,0.529328,100.000000
SPY_Call_346.0_19_SPY,0.430599,36.880001
SPY_Call_310.0_14_SPY,0.022410,71.910004
SPY_Call_408.0_7_SPY,0.009380,0.040000
SPY_Call_389.0_2_SPY,0.005372,0.100000
...,...,...
SPY_Put_438.0_3_SPY,0.000003,57.599998
SPY_Put_398.0_2_SPY,0.000003,17.190001
SPY_Call_437.0_6_SPY,0.000002,0.010000
SPY_Put_625.0_14_SPY,0.000001,244.250000


In [24]:
backtester.historicalTrajectory[timestamps[0]]["SPY"]

tensor([[380.8200, 383.7600, 379.3800, 388.0800, 387.8600, 390.5800, 395.5200,
         396.9600, 398.5000, 397.7700, 391.4900, 388.6400, 395.8800, 400.6300,
         400.2000, 400.3500, 404.7500, 405.6800, 400.5900, 406.4800]],
       dtype=torch.float64)

In [20]:
weightsDict

{'Deposit_GIC': {'weight': 0.5293284058570862, 'price': 100.0},
 'SPY_Stock_SPY': {'weight': 2.2618487491854466e-05,
  'price': 380.82000732421875},
 'SPY_Stock Short_SPY': {'weight': 9.94274887489155e-05,
  'price': 380.82000732421875},
 'SPY_Call_310.0_14_SPY': {'weight': 0.02240992709994316,
  'price': 71.91000366210938},
 'SPY_Put_310.0_14_SPY': {'weight': 3.454135367064737e-05,
  'price': 0.05999999865889549},
 'SPY_Call_389.0_2_SPY': {'weight': 0.005371840205043554,
  'price': 0.10000000149011612},
 'SPY_Put_389.0_2_SPY': {'weight': 6.981172191444784e-06,
  'price': 8.329999923706055},
 'SPY_Call_410.0_8_SPY': {'weight': 4.321640153648332e-05,
  'price': 0.07999999821186066},
 'SPY_Put_410.0_8_SPY': {'weight': 4.969092697137967e-05,
  'price': 29.280000686645508},
 'SPY_Call_332.0_5_SPY': {'weight': 2.2982821974437684e-05,
  'price': 49.400001525878906},
 'SPY_Put_332.0_5_SPY': {'weight': 2.1047906557214446e-05,
  'price': 0.019999999552965164},
 'SPY_Call_371.0_4_SPY': {'weight'

In [13]:
weights

tensor([5.2933e-01, 2.2618e-05, 9.9427e-05, 2.2410e-02, 3.4541e-05, 5.3718e-03,
        6.9812e-06, 4.3216e-05, 4.9691e-05, 2.2983e-05, 2.1048e-05, 6.4464e-06,
        2.6193e-05, 1.0794e-05, 2.7767e-05, 2.3315e-05, 2.7889e-05, 4.0006e-05,
        4.2876e-05, 7.5308e-05, 8.1542e-05, 6.8696e-05, 1.0418e-05, 2.9028e-05,
        2.5549e-05, 3.4931e-05, 7.6347e-06, 4.5672e-06, 5.5354e-05, 2.9248e-05,
        5.9478e-05, 9.9316e-06, 3.8736e-05, 1.3015e-06, 1.3884e-05, 7.0110e-06,
        3.1169e-06, 9.3803e-03, 9.2024e-05, 4.3060e-01, 5.3508e-05, 9.0177e-05,
        7.0995e-06, 3.4131e-05, 5.1016e-05, 2.1556e-06, 1.5671e-05, 1.0577e-04,
        5.1070e-05, 2.7025e-05, 4.2081e-05, 2.6067e-05, 1.2126e-04, 1.8453e-05,
        6.1491e-05, 3.4152e-05, 5.2631e-05, 2.9328e-05, 4.8682e-05, 3.8544e-05,
        7.5953e-05, 8.2679e-05, 7.6402e-05, 1.1390e-05, 8.2240e-05, 8.5230e-05,
        8.5578e-05, 3.5286e-05, 1.1269e-05, 9.4823e-06, 3.4827e-05, 6.3750e-05,
        1.0054e-05, 4.7769e-05, 2.5196e-

In [12]:
backtester.historicalTrajectory

{'SPY': {Timestamp('2023-01-03 00:00:00'): array([380.82000732, 383.76000977, 379.38000488, 388.07998657,
         387.85998535, 390.57998657, 395.51998901, 396.95999146,
         398.5       , 397.76998901, 391.48999023, 388.64001465,
         395.88000488, 400.63000488, 400.20001221, 400.3500061 ,
         404.75      , 405.67999268, 400.58999634, 406.48001099]),
  Timestamp('2023-01-04 00:00:00'): array([383.76000977, 379.38000488, 388.07998657, 387.85998535,
         390.57998657, 395.51998901, 396.95999146, 398.5       ,
         397.76998901, 391.48999023, 388.64001465, 395.88000488,
         400.63000488, 400.20001221, 400.3500061 , 404.75      ,
         405.67999268, 400.58999634, 406.48001099, 410.79998779]),
  Timestamp('2023-01-05 00:00:00'): array([379.38000488, 388.07998657, 387.85998535, 390.57998657,
         395.51998901, 396.95999146, 398.5       , 397.76998901,
         391.48999023, 388.64001465, 395.88000488, 400.63000488,
         400.20001221, 400.3500061 , 404.7